In [1]:
from tkinter import *
import tkinter as tk
from tkinter.filedialog import askopenfilename
from pm4py.visualization.dfg import visualizer as dfg_visualizer
import os
from PIL import Image, ImageTk
import pm4py
import pandas as pd


### função que printa o caminho 

In [2]:
def escolhe_arquivo():
    caminho_do_arquivo = askopenfilename(filetypes=[("Arquivo CSV", "*.csv"), ("Arquivo Excel", "*.xlsx"), ('só para testes PNG', '*.png'), ('xes', '*.xes')])
    if caminho_do_arquivo:
        
        label_arquivo.config(text=f"Arquivo selecionado: {caminho_do_arquivo}") # Calma é pra não estar definido msm
        #Mudar a chama dessa função quando for criar a função que trata os dados
        cria_grafo_dfg(caminho_do_arquivo)

In [3]:
def cria_grafo_dfg(caminho):
    #Botar uma condição para identificar o tipo do arq, se nn for xlsx vai dar problema

    log = pm4py.format_dataframe(pd.read_excel(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

    log['ID'] = log['ID'].astype(str)

    dfg, start_activities, end_activities = pm4py.discover_dfg(log)
    grafo = dfg_visualizer.apply(dfg, log=log)
    dfg_visualizer.save(grafo, "dfg_grafico.png")
    
    ##----------------------------------------------------------------------------------------------------------------------------------------

    print("Imagem do DFG salva como 'dfg_grafico.png'")

    

    exibe_grafico("dfg_grafico.png")

### Função que vai plotar os gráficos

In [4]:
def exibe_grafico(caminho):
    
    imagem_grafico = PhotoImage(file=caminho, master=frame_grafico)
    imagem_grafico = imagem_grafico.subsample(1,1)
    fig_grafico = Label(frame_grafico, image=imagem_grafico)
    fig_grafico.image = imagem_grafico
    fig_grafico.pack()

### INSTANCIANDO A JANELA

In [5]:
raiz = Tk(className='Primeira janela de teste para a interface')
raiz.geometry('960x540')

''

### Criando o frame de seleção de arquivos

In [6]:
frame1 = Frame(raiz) #bg="blue"
frame1.pack(side='top',fill=tk.BOTH)

### Botão de seleção de arquivos ###

In [7]:
arq = PhotoImage(file='iconArq.png', master=frame1)
arq = arq.subsample(2,2)
fig1 = Label(frame1, image=arq)
fig1.image = arq

botao = Button(frame1, image=arq, command=escolhe_arquivo, bd=0, highlightthickness=0, bg= 'green')
botao.pack()

label_arquivo = Label(frame1, text="Nenhum arquivo selecionado")
label_arquivo.pack(pady=10)

### Criando o frame Lateral

In [8]:
# def on_scroll(*args):
#     canvas.yview(*args)


frame_lateral = Frame(raiz)
frame_lateral.pack(side = 'right',pady=20, fill=tk.BOTH)

# Criando A lable na lateral direita
lable_direita = Label(frame_lateral, text="lateral esquerda", padx=40, pady=10, borderwidth=2, relief="solid")
lable_direita.pack(fill=tk.Y, expand=True)




# scale = tk.Scrollbar(frame_lateral, orient=tk.VERTICAL)
# scale.pack()



### frame do grafico

In [9]:
frame_grafico = Frame(raiz)

frame_grafico.pack()

### LOOPING DA JANELA PRINCIPAL

In [10]:
raiz.mainloop()

C:\Users\meduarda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pm4py\utils.py:96: UserWarning: Some rows of the Pandas data frame have been removed because of empty case IDs, activity labels, or timestamps to ensure the correct functioning of PM4Py's algorithms.
  warnings.warn("Some rows of the Pandas data frame have been removed because of empty case IDs, activity labels, or timestamps to ensure the correct functioning of PM4Py's algorithms.")


Imagem do DFG salva como 'dfg_grafico.png'
